RAJ RAUNAK KUMAR 20HCS4148

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# Digits dataset ko load karna
digits = load_digits()  # Digits dataset ko load karna
X = digits.data  # Features (images as flattened arrays)
y = digits.target  # Labels (digits 0-9)

# Dataset ko 70-30 ratio mein training aur testing sets mein divide karna
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  # 70-30 split

# Features ko normalize karna (for numerical stability)
X_train = X_train / 16  # Normalize to 0-1
X_test = X_test / 16  # Normalize to 0-1


In [2]:
# Single hidden layer neural network ka class
class TwoLayerANN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size  # Input size
        self.hidden_size = hidden_size  # Hidden layer size
        self.output_size = output_size  # Output size
        self.learning_rate = learning_rate  # Learning rate

        # Randomly initialize weights and biases
        self.hidden_weights = np.random.randn(input_size, hidden_size)  # Weights for hidden layer
        self.hidden_bias = np.zeros(hidden_size)  # Bias for hidden layer

        self.output_weights = np.random.randn(hidden_size, output_size)  # Weights for output layer
        self.output_bias = np.zeros(output_size)  # Bias for output layer

    def relu(self, x):
        # Relu activation function
        return np.maximum(0, x)

    def softmax(self, x):
        # Softmax activation function
        exps = np.exp(x - np.max(x))  # Stabilize with subtracting max
        return exps / np.sum(exps, axis=1, keepdims=True)

    def forward(self, X):
        # Forward pass through the network
        self.hidden_layer = self.relu(np.dot(X, self.hidden_weights) + self.hidden_bias)  # Hidden layer
        self.output_layer = self.softmax(np.dot(self.hidden_layer, self.output_weights) + self.output_bias)  # Output layer
        return self.output_layer

    def backward(self, X, y):
        # Backward pass for gradients
        m = X.shape[0]  # Number of samples

        # Derivative of softmax cross-entropy loss
        output_error = self.output_layer  # Copy output
        output_error[range(m), y] -= 1  # Subtract correct class (1-hot-like)

        # Gradients for output layer
        dW_out = np.dot(self.hidden_layer.T, output_error) / m
        dB_out = np.sum(output_error, axis=0) / m

        # Gradients for hidden layer
        hidden_error = np.dot(output_error, self.output_weights.T) * (self.hidden_layer > 0)  # Relu derivative
        dW_hidden = np.dot(X.T, hidden_error) / m
        dB_hidden = np.sum(hidden_error, axis=0) / m

        # Update weights and biases
        self.output_weights -= self.learning_rate * dW_out
        self.output_bias -= self.learning_rate * dB_out
        self.hidden_weights -= self.learning_rate * dW_hidden
        self.hidden_bias -= self.learning_rate * dB_hidden

    def train(self, X, y, epochs=10, batch_size=32):
        # Training the neural network
        m = X.shape[0]  # Total samples
        for _ in range(epochs):
            for i in range(0, m, batch_size):  # Mini-batches
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                self.forward(X_batch)  # Forward pass
                self.backward(X_batch, y_batch)  # Backward pass


In [3]:
# ANN model ko initialize karna with specific sizes
ann = TwoLayerANN(input_size=64, hidden_size=32, output_size=10, learning_rate=0.01)  # Input, hidden, output size

# ANN model ko train karna
ann.train(X_train, y_train, epochs=20, batch_size=32)  # Train for 20 epochs

# Testing set par predictions generate karna
y_pred = ann.forward(X_test)  # Forward pass for testing data
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax to class labels

# Accuracy ko calculate karna
test_accuracy = accuracy_score(y_test, y_pred_classes)  # Accuracy on test set
print(f"Test Accuracy: {test_accuracy:.2f}")  # Print test accuracy


Test Accuracy: 0.63
